In [1]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.opengl)


TensorFrost module loaded!


In [2]:

dt = 0.0001
ParticleN = 20000

IC_Radius = 5.0

def Force(x1, x2, v1, v2, dist):
    dx = x2 - x1
    dv = v2 - v1
    N = dx.shape[0]
    #dist = tf.norm(r)
    #dist = tf.reshape(dist, [N, N, 1])
    weight = tf.exp(-dist / 0.015)
    return 0.2 * dx / (dist ** 3.0 + 5e-4) #+ 0.1 * dv * weight#  - 50.0 * weight * dx / (dist*dist + 1e-5)

def n_body():
    X = tf.input([-1, 2], tf.float32)
    N = X.shape[0]
    V = tf.input([N, 2], tf.float32)

    i, j, _ = tf.indices([N, N, 1])
    dist = tf.sqrt((X[i, 0] - X[j, 0]) ** 2.0 + (X[i, 1] - X[j, 1]) ** 2.0)

    i, j, k = tf.indices([N, N, 2])
    
    Fij = Force(X[i, k], X[j, k], V[i, k], V[j, k], dist)
    Fij = tf.select(i == j, 0.0, Fij)
    
    Fi = tf.sum(Fij, axis=1)

    Vnew = V + Fi * dt
    Xnew = X + Vnew * dt

    return [Xnew, Vnew]

nbody = tf.compile(n_body)

n_body:
  Kernel count: 1
  Intermediate buffers: 0
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 414
  IR Compile time: 2.565900 ms
  Compiler time: 4769.993652 ms



In [3]:
def render_particles():
    X = tf.input([-1, 2], tf.float32)
    N = X.shape[0]
    params = tf.input([2], tf.int32)
    ImageW = params[0]
    ImageH = params[1]
    atomic_canvas = tf.zeros([ImageH, ImageW], tf.int32)

    i, = tf.indices([N])
    xpos = (X[i, 0] / IC_Radius) * tf.float(ImageH) + tf.float(ImageH) / 2.0
    ypos = (X[i, 1] / IC_Radius) * tf.float(ImageH) + tf.float(ImageW) / 2.0
    is_inside = (xpos >= 0.0) & (xpos < tf.float(ImageH)) & (ypos >= 0.0) & (ypos < tf.float(ImageW))
    
    with tf.if_cond(is_inside):
        xi = tf.int(xpos)
        yi = tf.int(ypos)
        xf = xpos - tf.float(xi)
        yf = ypos - tf.float(yi)
        w00 = (1.0 - xf) * (1.0 - yf)
        w01 = xf * (1.0 - yf)
        w10 = (1.0 - xf) * yf
        w11 = xf * yf
        tf.scatterAdd(atomic_canvas[xi, yi], tf.int(w00 * 255.0))
        tf.scatterAdd(atomic_canvas[xi, yi + 1], tf.int(w01 * 255.0))
        tf.scatterAdd(atomic_canvas[xi + 1, yi], tf.int(w10 * 255.0))
        tf.scatterAdd(atomic_canvas[xi + 1, yi + 1], tf.int(w11 * 255.0))

    i, j, ch = tf.indices([ImageH, ImageW, 3])
    image = tf.float(atomic_canvas[i, j]) / 500.0
    return [image]

render = tf.compile(render_particles)

render_particles:
  Kernel count: 3
  Intermediate buffers: 1
  Host readbacks: 2
  Host writes: 0
  Lines of generated code: 418
  IR Compile time: 5.664300 ms
  Compiler time: 4906.783691 ms



In [4]:
import time

X = np.random.randn(ParticleN, 2).astype(np.float32)
#V = np.zeros([N, 2], np.float32)
#initialize with rotation
V = np.zeros([ParticleN, 2], np.float32)
V[:, 0] = -X[:, 1] * 30.1
V[:, 1] = X[:, 0] * 30.1

Xtf = tf.tensor(X)
Vtf = tf.tensor(V)

ImageW = 1280
ImageH = 800
tf.show_window(ImageW, ImageH, "N-body simulation")

init_time = time.time()

while not tf.window_should_close():
    ImageW, ImageH = tf.get_window_size()
    cur_time = time.time() - init_time
    time_tf = tf.tensor(np.array([cur_time], np.float32))
    Xtf, Vtf = nbody(Xtf, Vtf)
    params = tf.tensor(np.array([ImageW, ImageH], np.int32))
    frame_tf, = render(Xtf, params)
    tf.render_frame(frame_tf)
    render_time = time.time() - init_time - cur_time
    tf.imgui_text("Simulation time: %.3f ms" % (render_time * 1000))
    tf.imgui_text("Particles: %d" % ParticleN)
    tf.imgui_text("FPS: %.1f" % (1.0 / (render_time + 1e-6)))
    tf.imgui_text("Time step: %.3f" % dt)


tf.hide_window()